## Positive Point Mutual Information

Point Mutual Information (PMI) 는 (word, contexts) 이나 (input, outputs) 와의 상관성을 측정하는 방법입니다. 두 변수 $x$, $y$ 의 상관성은 다음처럼 정의됩니다. 서로 상관이 없는 변수 $x$, $y$의 pmi 는 0 이며, 그 값이 클수록 positive correlated 있습니다. 

- $pmi(x,y) = log \left( \frac{p(x,y)}{p(x) \times p(y)} \right )$

Positive PMI 는 음의 값을 지니는 PMI 를 모두 0으로 치환합니다. 

- $ppmi(x,y) = max(0, log \left( \frac{p(x,y)}{p(x) \times p(y)} \right)$

그런데 PMI 는 infrequent $y$ 에 대하여 그 값이 지나치게 예민합니다. 이를 보완하기 위하여 smoothing 을 할 수 있습니다. soynlp 에서는 다음과 같은 smoothing 방법을 이용합니다. $\alpha$ 를 $p(y)$ 에 더합니다. 

- $pmi(x,y) = log \left( \frac{p(x,y)}{p(x) \times \left( p(y) + \alpha \right)} \right)$

$\alpha$ 는 y 의 threshold 역할을 합니다. PMI 는 다음처럼 기술될 수 있습니다. $p(y)$ 가 $\alpha$ 보다 큰 값들이 positive pmi value 를 지닐 수 있습니다. 

- $pmi(x,y) = \frac{p(y \vert x)}{\left( p(y) + \alpha \right)}$

PPMI 를 위해서 min_pmi 를 기준으로 threshold cutting 을 하는 기능도 제공합니다. 

## Word - context matrix

Word - context 그래프는 단어의 문맥을 파악하기 위해 사용될 수 있습니다. 

sentence = ['a', 'little', 'cat', 'sit', 'on', 'table'] , window = 2 일 때, 'cat' 의 context words 는 ['a', 'little', 'sit', 'on'] 입니다. sent_to_word_context_matrix() 함수는 이 역할을 수행합니다. min_tf 는 minimum frequency 입니다. 

Return 은 scipy.sparse.csr.csr_matrix 형식의 word - context matrix 와 list of str 형식의 vocabulary list 입니다. 

In [1]:
import sys
sys.path.append('../')

import soynlp
print(soynlp.__version__)

0.0.49


사용할 토크나이저를 학습합니다. 

In [2]:
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

corpus_path = '2016-10-20.txt'
corpus = DoublespaceLineCorpus(corpus_path, iter_sent = True)
print('num sents = {}'.format(len(corpus)))

word_extractor = WordExtractor()
word_extractor.train(corpus)
cohesions = word_extractor.all_cohesion_scores()
print(cohesions['뉴스'])

l_cohesions = {word:score[0] for word, score in cohesions.items()}
tokenizer = LTokenizer(l_cohesions)
print(tokenizer('하루의 뉴스를 학습했습니다'))

num sents = 223357
training was done. used memory 0.724 Gbse memory 0.777 Gb
all cohesion probabilities was computed. # words = 223348
(0.487322733132789, 0.22771099423991986)
['하루', '의', '뉴스', '를', '학습', '했습니다']


sent_to_word_contexts_matrix() 에 window, min_tf, tokenizer 를 넣습니다. verbose=True 이면 vectorizing 되는 상태의 모니터링이 가능합니다. 

In [3]:
from soynlp.vectorizer import sent_to_word_contexts_matrix

x, idx2vocab = sent_to_word_contexts_matrix(
    corpus,
    windows = 3,
    min_tf = 10,
    tokenizer = tokenizer, # (default) lambda x:x.split(),
    dynamic_weight = False,
    verbose = True)

Create (word, contexts) matrix
  - counting word frequency from 223356 sents, mem=0.726 Gb
  - scanning (word, context) pairs from 223356 sents, mem=1.184 Gb
  - (word, context) matrix was constructed. shape = (48583, 48583)                    
  - done


## PMI

soynlp.word.pmi 는 x 의 (rows, columns) 에 대한 pmi 를 계산합니다. row 가 x, column 이 y 입니다. 

In [4]:
from soynlp.word import pmi as pmi_func

pmi, px, py = pmi_func(
    x,
    min_pmi = 0,
    alpha = 0.0,
    beta = 0.75
)

단어 `아이오아이` 와 pmi 가 높은 (`아이오아이` 주변에 자주 등장한) 단어를 찾습니다. 

In [5]:
vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
query = vocab2idx['아이오아이']

submatrix = pmi[query,:].tocsr() # get the row of query
contexts = submatrix.nonzero()[1] # nonzero() return (rows, columns)
pmi_i = submatrix.data

most_relateds = [(idx, pmi_ij) for idx, pmi_ij in zip(contexts, pmi_i)]
most_relateds = sorted(most_relateds, key=lambda x:-x[1])[:10]
most_relateds = [(idx2vocab[idx], pmi_ij) for idx, pmi_ij in most_relateds]

from pprint import pprint
pprint(most_relateds)

[('신용재', 7.141024243363501),
 ('완전체로', 7.015931333051887),
 ('후크송으로', 6.977172538792954),
 ('세븐', 6.830966065152642),
 ('오블리스', 6.717087720696372),
 ('신용재가', 6.670209432230605),
 ('정채연은', 6.560811696929447),
 ('상큼한', 6.334895661765805),
 ('선배님들이', 6.154353835218832),
 ('금빛나', 5.878560250124844)]


단어 `아이오아이` 와 유사한 contexts vector 를 지닌 단어를 찾습니다. 

In [6]:
from soynlp.utils import most_similar

most_similar('아이오아이', pmi, vocab2idx, idx2vocab)

[('엠카운트다운', 0.2510761472706349),
 ('너무너무', 0.24962359995756267),
 ('신용재', 0.24854244222409871),
 ('갓세븐', 0.20190168259477226),
 ('컴백', 0.20040893958615869),
 ('완전체로', 0.1971543262212967),
 ('걸그룹', 0.19516713512611328),
 ('엠카', 0.1870115403151804),
 ('오블리스', 0.18510781764378215),
 ('다비치', 0.1827054015470695)]